# Aleasim Release 0.1 Demo

## Features in Version 0.1

### Kernel
- [x] Time and Reference Frame Management
- [x] Scheduler

### Analyser
- [x] Shared memory interface
- [x] Simple grahpics demo

### Models
- [x] Spacecraft - Parent Model
- [x] Spacecraft -Attitude Dynamics
- [x] Spacecraft - Orbital Dynamics
- [x] Magnetic Field Model
- [x] Gravity Model
- [x] Earth, Sun, Moon Ephimeris Models

## Critical Features to Implement - Future Releases
- [ ] Spacecraft - Magnetorquer Model
- [ ] Spacecraft - Reaction Wheel Model
- [ ] Spacecraft - Magnetometer Model
- [ ] Spacecraft - Coarse Sun Sensor Model
- [ ] Spacecraft - Gyroscope Model
- [ ] Spacecraft - IMU Model
- [ ] FSW - CDH / state machine model
- [ ] FSW - Attitude Estimator Models (QUEST, EKF, etc)
- [ ] FSW - Controller Models (PID, BDOT, etc)

## Nice to Haves
- [ ] GUI
- [ ] Better data representation
- [ ] Performance Improvements

In [ ]:
import alea.sim
import numpy as np
import scipy as sp
import skyfield

from datetime import datetime
import time
from alea.sim.epa.magnetic_field_model import EarthMagneticFieldModel
from alea.sim.epa.attitude_dynamics import AttitudeDynamicsModel
from alea.sim.epa.orbit_dynamics import OrbitDynamicsModel
from alea.sim.kernel.kernel import AleasimKernel

from alea.sim.spacecraft.spacecraft import Spacecraft
import matplotlib.pyplot as plt
import logging
import cProfile
import pstats
from pstats import SortKey
from matplotlib import pyplot as plt
import skyfield.sgp4lib

In [ ]:
logging.basicConfig(level=logging.INFO)

kernel = AleasimKernel(dt='0.01')
# kernel.scheduler._debug_realtime = True

adyn = AttitudeDynamicsModel(kernel)
adyn.set_state(np.array([1,0,0,0,25.0,3.0,1e-3]))
odyn = OrbitDynamicsModel(kernel)

magm = EarthMagneticFieldModel(kernel)

kernel.add_model(odyn, True)
kernel.add_model(adyn, True)
kernel.add_model(magm)

adyn._integrator_type = "quad"

kernel.advance(10)

kernel.kill()

In [ ]:
%load_ext autoreload
%autoreload 2

from alea.sim.epa.disturbance_model import DisturbanceModel
from alea.sim.math_lib import Quaternion

kernel = AleasimKernel(dt='0.01')
adyn = AttitudeDynamicsModel(kernel)
adyn.set_state(np.array([1,0,0,0,25.0,3.0,1e-3]))
odyn = OrbitDynamicsModel(kernel)
magm = EarthMagneticFieldModel(kernel)
DModel = DisturbanceModel(kernel)

kernel.add_model(odyn, True)
kernel.add_model(adyn, True)
kernel.add_model(magm)
kernel.add_model(DModel)

kernel.step()

q_sample = Quaternion(adyn._state[0:4])
print(odyn.calculate_sun_vector())
print(q_sample)
print("dist = ",DModel.get_T_disturbance(q_sample))


In [ ]:
kernel.advance(1)

n = adyn.arr_size

objs = plt.plot(adyn.state_array[:n,0:4])
plt.title("Quaternion Elements")
plt.legend(iter(objs), ('q0', 'q1', 'q2', 'q3'))
plt.show()

objs = plt.plot(adyn.state_array[:n,4:7])
plt.title("Angular Rates")
plt.legend(iter(objs), ('w1', 'w2', 'w3'))
plt.show()

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(odyn.state_array[:n,0],odyn.state_array[:n,1], odyn.state_array[:n,2])
ax.set_xlabel('X ECI [m]')
ax.set_ylabel('Y ECI [m]')
ax.set_zlabel('Z ECI [m]')
plt.show()

In [ ]:
import math
#position in lat, lon and alt
lla = odyn.position_lonlat

#lla returns radians (for now)
lla[0] = math.degrees(lla[0])
lla[1] = math.degrees(lla[1])

#get the mag vector
mag_ned = magm.get_mag_vector(lla)

#transform it from ned to eci
mag_eci = kernel.eci_frame.transform_vector_from_frame(mag_ned, kernel.ned_frame)
mag_body = kernel.body_frame.transform_vector_from_frame(mag_ned, kernel.ned_frame)

print(f'mag eci 1: {mag_eci}')

print(f'mag body: {mag_body}')

#body quaternion (eci->body frame)
adyn.quaternion